# eRADAR 

In [0]:
import numpy as np
import pandas as pd

Load the patient data.
- /Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_2025.csv

In [0]:
pt_data = pd.read_csv("/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_2025.csv", header=0, index_col=0)
pt_data.head()

Apply the eRADAR model to the patient data.

In [0]:
class eRADAR:
    def __init__(self):
        self.intercept = -11.83
        self.weights = [0.11, -0.10, 0.28, 0.18, 0.06, 0.34, -0.11, 0.05, -0.15, 0.01, -0.17, -0.05, -0.34, -0.07, -0.25, 0.46, 0.03, 0.20, -0.15, 0.03, 0.24, 0.29, -0.11, -0.09, -0.55, 0.34, 0.00, -0.13, 0.47, 0.58, 0.03]

    def score(self, x):
        return 1 / (1 + np.exp(-self.intercept - np.dot(self.weights, x)))

In [0]:
model = eRADAR()

pt_scores = pt_data[["age", "female", "dx_chf", "dx_cebv", "dx_diab", "dx_diab_c", "dx_cpd", "dx_hypothyoidism", "dx_rf", "dx_lymphoma", "dx_solidtumor", "dx_ra", "dx_weightloss", "dx_fluid", "dx_anemia", "dx_bipolar", "dx_depression", "dx_trauma", "dx_tobacco", "dx_atrial", "dx_gait", "low_bmi", "high_bmi", "hbp", "op_primary_care_visit", "ed_visit", "home_visit", "pt_visit", "cognitive_visit", "med_antidepressant", "med_sleepaid"]].apply(model.score, axis=1)

Analysis.

In [0]:
import matplotlib.pyplot as plt

In [0]:
print("Median [IQR] = %.3f [%.3f - %.3f]" % (np.median(pt_scores), np.quantile(pt_scores, 0.25), np.quantile(pt_scores, 0.75)))

In [0]:
qs = [99] + list(range(95, 45, -5))
thresholds = np.percentile(pt_scores, qs)
n_patients = [(pt_scores >= t).sum() for t in thresholds]
ages = [(round(pt_data.loc[(pt_scores >= t), "age"].mean()), round(pt_data.loc[(pt_scores >= t), "age"].std(), 2)) for t in thresholds]
females = [round(pt_data.loc[(pt_scores >= t), "female"].sum() / pt_data.loc[(pt_scores >= t)].shape[0], 2) for t in thresholds]

penn_data = pd.DataFrame({"Cutoff Percentile": qs, "Cutoff": thresholds, "Identified": n_patients, "Age (mean, std)": ages, "Pct. Female": females})    # note "Identified" is the number of patients to recruit from
penn_data

In [0]:
plt.figure(figsize=(15, 5))
plt.hist(pt_scores, bins=500)

for i in [0, 1, 2, 4, 9]:
    plt.axvline(thresholds[i], color="red", linestyle="--", alpha=0.6)
    plt.text(thresholds[i], 1.02, f"{qs[i]}th", transform=plt.gca().get_xaxis_transform(), color="red", rotation=90, ha="center")

plt.xlim([0, 1])
plt.xticks(np.arange(0.0, 1.1, 0.1))
plt.yscale("log")
plt.xlabel("eRADAR Score")
plt.ylabel("Frequency (Num. Patients)")
plt.title("Distribution of eRADAR Scores for Penn Patients", y=1.05)
plt.grid()
plt.show()

Comparing number of recruits at varing eRADAR score cutoffs across cohorts.

In [0]:
N = 688599
N_positive = round((N * 11.1) / 1000)
act = {
    "Cutoff Percentile": [99, 95, 90, 85, 75],
    "Sensitivity": [6.5, 22.5, 36.2, 47.1, 65.9],
    "Specificity": [99.4, 96.4, 91.6, 87.2, 78.0],
    "PPV": [23.7, 15.7, 11.5, 10.0, 8.3]
}

act_data = pd.DataFrame(act)
act_data["TP"] = (act_data["Sensitivity"] / 100) * N_positive
act_data["Identified"] = act_data["TP"] / (act_data["PPV"] / 100)
act_data

In [0]:
N = 688599
N_positive = round((N * 11.1) / 1000)
kpwa = {
    "Cutoff Percentile": [99, 95, 90, 85, 75],
    "Sensitivity": [9.5, 35.2, 54.3, 65.9, 79.6],
    "Specificity": [99.2, 95.3, 89.9, 84.6, 74.4],
    "PPV": [10.1, 6.5, 4.8, 3.8, 2.8]
}

kpwa_data = pd.DataFrame(kpwa)
kpwa_data["TP"] = (kpwa_data["Sensitivity"] / 100) * N_positive
kpwa_data["Identified"] = kpwa_data["TP"] / (kpwa_data["PPV"] / 100)
kpwa_data

In [0]:
N = 47348
N_positive = round((N * 4.6) / 1000)
ucsf = {
    "Cutoff Percentile": [99, 95, 90, 85, 75],
    "Sensitivity": [7.1, 32.1, 44.4, 56.2, 73.6],
    "Specificity": [98.9, 94.9, 89.8, 84.8, 74.4],
    "PPV": [2.9, 2.8, 2.0 ,1.6, 1.3]
}

ucsf_data = pd.DataFrame(ucsf)
ucsf_data["TP"] = (ucsf_data["Sensitivity"] / 100) * N_positive
ucsf_data["Identified"] = (round(ucsf_data["TP"] / (ucsf_data["PPV"] / 100))).astype(int)
ucsf_data

In [0]:
temp = pd.merge(penn_data[["Cutoff Percentile", "Identified"]], act_data[["Cutoff Percentile", "Identified"]], on="Cutoff Percentile", how="inner")
temp.columns = ["Cutoff Percentile", "Us (Penn)", "ACT"]
temp = pd.merge(temp, kpwa_data[["Cutoff Percentile", "Identified"]], on="Cutoff Percentile", how="left")
temp.columns = ["Cutoff Percentile", "Us (Penn)", "ACT", "KPWA"]
temp = pd.merge(temp, ucsf_data[["Cutoff Percentile", "Identified"]], on="Cutoff Percentile", how="left")
temp.columns = ["Cutoff Percentile", "Us (Penn)", "ACT", "KPWA", "UCSF"]
round(temp)

Feature importance.